
<h1><center>Многоязычная классификация текстов на основе BERT</center></h1>

Мы продолжим знакомство с моделями на основе архитектуры Трансформер. 
На этом занятии мы рассмотрим классификатор на основе модели RoBERTa (модель на базе BERT), дообученный для решения задачи Natural Language Inference (NLI). В этой задаче нам требуется предсказать, если ли логическая связь между двумя предложениями: следует ли одного из другого.  Этот классификатор, предобученный на задаче NLI, позволит нам решать задачу классификации в Zero-shot постановке. В такой постановке модели НЕ требуется обучение на размеченных данных, поскольку любую задачу мы можем свести к задаче NLI –- мы рассмотрим, как именно в примерах ниже.

Мы загрузим заранее обученный классификатор и будем использовать его, чтообы классифицировать тексты на нужные нам классы. 

Два важных отличия рассматриваемой сегодня модели от тех, с которыми мы работали на предыдущих занятиях:
- Подход zero-shot не требует обучения модели, нам следует лишь подать классификатору текст и названия нужных нам классов. Модель будет строить предсказание на основе своего внутреннего представления, обученного заранее.
- Данная модель многоязычна. Мы будем работать с текстами, написанными на разных языках.

## Классификация текстов на английском языке

Установим библиотеку transformers и загрузим модуль pipeline, в котором находится нужная нам модель.

In [ ]:
!pip install transformers==3.1.0

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification")
# classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU


С помощью этой модели мы можем классифицировать тексты как принадлежащие или не принадлежащие одному из нужных нам классов.

По умолчанию предполагается, что текст принадлежит ровно к одному из объявленных классов, а модель возвращает вероятности каждого из классов, которые в сумме равны 1.

Посмотрим простой пример.

In [ ]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics"]

classifier(sequence, candidate_labels)

{'labels': ['politics', 'economics', 'public health'],
 'scores': [0.972518801689148, 0.01458414364606142, 0.012897025793790817],
 'sequence': 'Who are you voting for in 2020?'}

Модель также может решать задачу многоклассоовой классификации, определяя для каждого класса, принадлежит ли к нему текст или нет. 
В этом случае модель возвращает вероятности каждого из классов независимо. Нам понадобиться добавить парамерт ```multi_class=True``` в вызов функции. 

In [ ]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics", "elections"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['politics', 'elections', 'public health', 'economics'],
 'scores': [0.972069501876831,
  0.967610776424408,
  0.03248710557818413,
  0.0061644683592021465],
 'sequence': 'Who are you voting for in 2020?'}

Мы можем решать не только задачу классификации по темам, но и задачу классификации по тональности: можно классифицировать отзывы на негативный и позитивный классы.

In [ ]:
sequence = "I hated this movie. The acting sucked."
candidate_labels = ["positive", "negative"]

classifier(sequence, candidate_labels)

{'labels': ['negative', 'positive'],
 'scores': [0.9916268587112427, 0.00837317667901516],
 'sequence': 'I hated this movie. The acting sucked.'}


Модель, лежащая в основе данного классификатора, была обучена на задаче Natural Language Inference (NLI). Задача состояла в следующем: по двум входящим текстам требовалось определить, является ли один из них продолжением другого.

Такую модель можно использвать в задаче zero-shot классификации, если свести классификацию к задаче NLI. Для этого модели на вход подаются два текста:
- текст, который нужно классифицировать (*предпосылка*)
- текст шаблона, в который вставлено название нужного класса (*гипотеза*)


Если с точки зрения NLI-модели текст гипотезы продолжает текст предпосылки, то мы можем заключить, что классифицируемый текст относится к соответствующему классу.

По умолчанию заданные метки классов вставляются в шаблон `This example is {class_name}.` 

Во многих случаях такой подход работает достаточно хорошо, но в некоторых задачах качество классификации можно улучшить, используя более подходящий шаблон.

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'labels': ['negative', 'positive'],
  'scores': [0.9916267991065979, 0.008373182266950607],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['negative', 'positive'],
  'scores': [0.8148515820503235, 0.1851484179496765],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

Второй пример выше несколько сложнее первого, поэтому модель предсказывает негативный класс менее уверенно. 

Попробуем повысить уверенность классификации, используя более подходящий шаблон для данной задачи. Вместо шаблона по умолчанию `This example is {}.`, мы будем использовать `The sentiment of this review is {}.` (здесь `{}` будет заменено на название класса)

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]


Используя более точные и более подходящие к нашему контексту шаблоны, мы можем получить более точную классификацию.

## Многоязычная классификация

Данный классификатор можно использовать не только для английского языка, но и для ряда других. Для этого была обучена кросс-язычная модель на базе модели XLM RoBERTa.

Чтобы использовать эту модель, достаточно добавить дополнительный параметр ```model``` при объявлении классификатора.

In [ ]:
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')



Модель позволяет использовать любую комбинацию языков. Можно, например, классифицировать русский текст с английскими названиями классов.

In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics"]

classifier(sequence, candidate_labels)

{'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9048484563827515, 0.05722189322113991, 0.03792969882488251],
 'sequence': 'За кого вы голосуете в 2020 году?'}

Теперь можно сделать то же самое, только поменять названия классов на французские.

In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9726154804229736, 0.017128489911556244, 0.010256024077534676],
 'sequence': 'За кого вы голосуете в 2020 году?'}


Как было отмечено выше, по умолчанию в модели используется шаблон на английском языке, `This text is {}.`
В случае, если мы работаем с другим языком, имеет смысл поменять данный шаблон на аналогичный, написанный на нужном нам языке.

In [ ]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9109585881233215, 0.05954807624220848, 0.029493311420083046],
 'sequence': '¿A quién vas a votar en 2020?'}

Модель была дообучена на корпусе XNLI, в состав которого входят тексты на 15 языках: английский, арабский, болгарский, вьетнамский, греческий, испанский, немецкий, русский, суахили, тайский, турецкий, урду, французский и хинди. 

Базовая модель RoBERTa, на основе которой была построена данная модель, была обучена на 100 языках, поэтому для каждого из этих языков эта модель будет также работать в некоторой степени.  

## Итоги

Сегодня мы познакомились с моделью на базе RoBERTa для решения задачи NLI.

Мы научились использовать построенный на основе данной модели классификатор для классификации текстов на разных языках по разным классам, названия которых мы задавали сами.

На следующих занятиях мы продолжим изучение моделей, основанных на архитектуре BERT и рассмотрим другие задачи, которые можем решать с их помощью.